A company is organizing a meeting and has a list of n employees, waiting to be invited. They have arranged for a large circular table, capable of seating any number of employees.

The employees are numbered from 0 to n - 1. Each employee has a favorite person and they will attend the meeting only if they can sit next to their favorite person at the table. The favorite person of an employee is not themself.

Given a 0-indexed integer array favorite, where favorite[i] denotes the favorite person of the ith employee, return the maximum number of employees that can be invited to the meeting.

 

Example 1:


Input: favorite = [2,2,1,2]
Output: 3
Explanation:
The above figure shows how the company can invite employees 0, 1, and 2, and seat them at the round table.
All employees cannot be invited because employee 2 cannot sit beside employees 0, 1, and 3, simultaneously.
Note that the company can also invite employees 1, 2, and 3, and give them their desired seats.
The maximum number of employees that can be invited to the meeting is 3. 
Example 2:

Input: favorite = [1,2,0]
Output: 3
Explanation: 
Each employee is the favorite person of at least one other employee, and the only way the company can invite them is if they invite every employee.
The seating arrangement will be the same as that in the figure given in example 1:
- Employee 0 will sit between employees 2 and 1.
- Employee 1 will sit between employees 0 and 2.
- Employee 2 will sit between employees 1 and 0.
The maximum number of employees that can be invited to the meeting is 3.
Example 3:


Input: favorite = [3,0,1,4,1]
Output: 4
Explanation:
The above figure shows how the company will invite employees 0, 1, 3, and 4, and seat them at the round table.
Employee 2 cannot be invited because the two spots next to their favorite employee 1 are taken.
So the company leaves them out of the meeting.
The maximum number of employees that can be invited to the meeting is 4.
 

Constraints:

n == favorite.length
2 <= n <= 105
0 <= favorite[i] <= n - 1
favorite[i] != i

엄청나게 Hard 한 난이도의 graph 문제다
depth first search graph topological sort의 합작품.

In [13]:
test  = [5,3,1,145]
# test.index(3)
len(test)-test.index(3)
test[-2]

1

In [10]:
s = {1}
type(s)
1 in s
len(s)
s.add(5)
s.add(5)
s.add(5)
s

{1, 5}

In [15]:
# 내 투박했던 solution
class Solution:
    def maximumInvitations(self, favorite: List[int]) -> int:
        ans = 0
        m = {}
        n = len(favorite)
        
        for idx, val in enumerate(favorite):
            m[idx] = val
        
        for i in range(n):
            visited = [i]
            val = favorite[i]
            while val not in visited :
                visited.append(val)
                val = m[val]
            # 중간에 visited 에서 cycle이 발생한 경우
            # 1, make a cycle from i
            if i == val : ans = max(ans,len(visited))
            # 2, make a cycle not from i and no 2-component cycle
            elif val != visited[-2] : ans = max(ans,len(visited)-visited.index(val))
            # 3, make a cycle not from i and 2-component cycle
            else : ans = max(ans,len(visited))
            
        return ans
    # test case를 다 통과하진 못했다

NameError: name 'List' is not defined

In [16]:
# time excced 가 걸린 풀이
class Solution:
    def maximumInvitations(self, favorite: List[int]) -> int:
        n = len(favorite)
        visited = [-1] * n  # 방문 상태 (0: 미방문, 1: 방문 중, 2: 방문 완료)
        depth = [-1] * n    # 각 노드에서의 깊이
        max_cycle_len = 0   # 일반 사이클의 최대 길이
        max_tail_sum = 0    # 2-Component Cycle에서의 최대 꼬리 길이 합
        
        # DFS를 사용하여 사이클 길이 계산
        def dfs(node, d):
            if visited[node] == 1:  # 사이클 탐지
                return d - depth[node]
            if visited[node] == 2:  # 이미 방문 완료된 노드
                return 0
            visited[node] = 1
            depth[node] = d
            cycle_length = dfs(favorite[node], d + 1)
            visited[node] = 2
            return cycle_length
        
        # 각 노드에 대해 DFS 수행
        for i in range(n):
            if visited[i] == -1:
                cycle_length = dfs(i, 0)
                max_cycle_len = max(max_cycle_len, cycle_length)
        
        # 2-Component Cycle 계산
        tail_lengths = [0] * n
        for i in range(n):
            if favorite[favorite[i]] == i and i < favorite[i]:
                # i와 favorite[i]가 서로 좋아하는 관계
                def find_tail(node, exclude):
                    max_len = 0
                    stack = [(node, 0)]
                    while stack:
                        current, length = stack.pop()
                        max_len = max(max_len, length)
                        for next_node in range(n):
                            if favorite[next_node] == current and next_node != exclude:
                                stack.append((next_node, length + 1))
                    return max_len
                
                tail1 = find_tail(i, favorite[i])
                tail2 = find_tail(favorite[i], i)
                max_tail_sum += tail1 + tail2 + 2
        
        return max(max_cycle_len, max_tail_sum)


In [ ]:
# 모범풀이
class Solution:
    def maximumInvitations(self, favorite: List[int]) -> int:
        adj=defaultdict(list)
        transpose=defaultdict(list)
        #first we apply kosaraju, its a cycle, it is that answer itself
        #or we can combine multiple 2 sized connected components (if 0's fav is 1 and 1s fav is 0) ->one exception case to consider
        #max of both the above conditions
        for i in range(len(favorite)):
            adj[i].append(favorite[i])
            transpose[favorite[i]].append(i)
        
        stack=[]
        visited=[False]*len(favorite)
        def dfs(node):
            visited[node]=True
            for n in adj[node]:
                if not visited[n]:
                    dfs(n)
            stack.append(node)
        
        for i in range(len(favorite)):
            if not visited[i]:
                dfs(i)
        
        sccs=[] #list of sets of each scc
        scc=set()
        visited=[False]*len(favorite)
        def traverseForScc(node):
            visited[node]=True
            scc.add(node)
            for n in transpose[node]:
                if not visited[n]:
                    traverseForScc(n)
        
        while stack:
            top = stack.pop()
            if not visited[top]:
                scc = set()
                traverseForScc(top)
                sccs.append(scc)
        
        ans=max([len(scc) if len(scc)!=2 else -1 for scc in sccs])
        #above was from kosaraju

        def findLongestArm(a,b):
            l=0
            for node in transpose[a]:
                if node!=b:
                    l=max(l,1+findLongestArm(node,b))
            return l

        twoNodeSccs = 0
        for n1,n2 in [scc for scc in sccs if len(scc)==2]:
            twoNodeSccs+= 2 + findLongestArm(n1,n2) + findLongestArm(n2,n1)
        return max(ans,twoNodeSccs)

In [ ]:
# C++
class Solution {
public:
    int maximumInvitations(vector<int>& fav) {
        int n = fav.size();
        vector<int> inDeg(n, 0), depth(n, 1);
        for (int i{0}; i < n; ++i) inDeg[fav[i]]++;
        queue<int> q;
        for (int i{0}; i < n; ++i) if (inDeg[i] == 0) q.push(i);
        while (!q.empty()) {
            int cur = q.front(); q.pop();
            int nxt = fav[cur];
            depth[nxt] = max(depth[nxt], depth[cur] + 1);
            if (--inDeg[nxt] == 0) q.push(nxt);
        }
        int maxCycle{0}, twoCycleSum{0};
        for (int i{0}; i < n; ++i) {
            if (inDeg[i] == 0) continue;
            int len{0}, cur = i;
            while (inDeg[cur] != 0) {
                inDeg[cur]=0;
                len++;
                cur = fav[cur];
            }
            if (len == 2) {
                twoCycleSum += depth[i] + depth[fav[i]];
            } else {
                maxCycle = max(maxCycle, len);
            }
        }
        return max(maxCycle, twoCycleSum);
    }
};
static auto init = []() { ios::sync_with_stdio(false); cin.tie(nullptr); cout.tie(nullptr); return nullptr; }();

In [ ]:
# # C
# bool visited[100000] = {};
# int dfs(int i, vector<vector<int>> &al) {
#     visited[i] = true;
#     return 1 + accumulate(begin(al[i]), end(al[i]), 0, [&](int res, int j){
#         return max(res, dfs(j, al));
#     });
# }
# int maximumInvitations(vector<int>& fav) {
#     int res = 0;
#     vector<vector<int>> al(fav.size());
#     for(int i = 0; i < fav.size(); ++i)
#         if (fav[fav[i]] != i)
#             al[fav[i]].push_back(i);
#     for(int i = 0; i < fav.size(); ++i)
#         if (fav[fav[i]] == i)
#             res += dfs(i, al);
#     for(int i = 0; i < fav.size(); ++i) {
#         int cnt = 0, j = i;
#         for (; visited[j] == false; j = fav[j]) {
#             visited[j] = true;
#             ++cnt;
#         }
#         for (int k = i; k != j; k = fav[k])
#             --cnt;
#         res = max(res, cnt); 
#     }
#     return res;
# }

In [ ]:
# C
struct node {
    int val;
    struct node* next;
};

struct node* createNode(int val) {
    struct node* ret = malloc(sizeof(struct node));
    ret->val = val;
    ret->next = NULL;

    return ret;
}

void addNode(struct node** map, int idx, int val) {
    struct node* tmp = map[idx]->next;
    map[idx]->next = createNode(val);
    map[idx]->next->next = tmp;
}

int bfs(struct node** map, int start, int n, int skip) {
    int vis[n];
    int queue[n][2];
    memset(vis, 0, sizeof(vis));

    int front = 0, rear = 0, maxDist = 0;
    
    vis[start] = vis[skip] = 1;
    queue[rear][0] = start;
    queue[rear++][1] = 0;
    
    while(front < rear) {
        int u = queue[front][0];
        int distU = queue[front][1];
        front++;
        
        for(struct node* tmp = map[u]->next; tmp; tmp = tmp->next) {
            int v = tmp->val;
            if(!vis[v]) {
                vis[v] = 1;
                int newDist = distU + 1;
                queue[rear][0] = v;
                queue[rear++][1] = newDist;
                maxDist = maxDist > newDist? maxDist : newDist;
            }
        }
    }
    
    return maxDist;
}

int maximumInvitations(int* favourite, int favouriteSize) {
    int* vis = calloc(favouriteSize, sizeof(int));
    struct node** map = malloc(favouriteSize * sizeof(struct node*));

    for(int i = 0; i < favouriteSize; i++) {
        map[i] = createNode(i);
    }
    for(int i = 0; i < favouriteSize; i++) {
        addNode(map, favourite[i], i);
    }
    
    int maxCycle = 0;
    int dCycle = 0;
    
    for(int i = 0; i < favouriteSize; i++) {
        if(!vis[i]) {
            int tmpVis[favouriteSize];
            memset(tmpVis, 0, sizeof(tmpVis));
            
            int curr = i;
            int dist = 0;
            
            while(!vis[curr]) {
                vis[curr] = 1;
                tmpVis[curr] = ++dist;      
                int next = favourite[curr];
                
                if(tmpVis[next] && tmpVis[next] < tmpVis[curr]) {
                    int cycleLength = dist - tmpVis[next] + 1;
                    maxCycle = maxCycle > cycleLength? maxCycle : cycleLength;
                    if(cycleLength == 2) {
                        dCycle += 2 + bfs(map, curr, favouriteSize, next) + bfs(map, next, favouriteSize, curr);
                    }
                    break;
                }       
                curr = next;
            }
        }
    }
    
    return (maxCycle > dCycle) ? maxCycle : dCycle;
}

In [ ]:
# topological sort
from collections import deque, defaultdict

class Solution:
    def maximumInvitations(self, favorite: List[int]) -> int:
        n = len(favorite)
        in_degree = [0] * n  # 각 노드의 진입 차수
        graph = defaultdict(list)  # 그래프 표현
        tail_length = [0] * n  # 각 노드의 꼬리 길이 저장
        
        # 그래프와 진입 차수 초기화
        for i in range(n):
            graph[favorite[i]].append(i)
            in_degree[favorite[i]] += 1
        
        # 위상 정렬을 위한 큐 초기화
        queue = deque()
        for i in range(n):
            if in_degree[i] == 0:
                queue.append(i)
        
        # 위상 정렬로 꼬리 길이 계산
        while queue:
            node = queue.popleft()
            next_node = favorite[node]
            tail_length[next_node] = max(tail_length[next_node], tail_length[node] + 1)
            in_degree[next_node] -= 1
            if in_degree[next_node] == 0:
                queue.append(next_node)
        
        # 사이클 길이 계산
        visited = [False] * n
        max_cycle_length = 0
        for i in range(n):
            if in_degree[i] > 0 and not visited[i]:  # 사이클에 속한 노드
                cycle_length = 0
                current = i
                while not visited[current]:
                    visited[current] = True
                    cycle_length += 1
                    current = favorite[current]
                max_cycle_length = max(max_cycle_length, cycle_length)
        
        # 2-Component Cycle 처리
        two_component_max = 0
        for i in range(n):
            if favorite[favorite[i]] == i and i < favorite[i]:  # 서로를 좋아하는 두 노드
                two_component_max += tail_length[i] + tail_length[favorite[i]] + 2
        
        # 최대값 반환
        return max(max_cycle_length, two_component_max)


In [ ]:
1. defaultdict(list)
특징:

defaultdict는 collections 모듈에서 제공하는 클래스입니다.
키가 없을 때 자동으로 초기값을 설정해줍니다.
defaultdict(list)는 기본적으로 빈 리스트 []를 초기값으로 설정합니다.

3. 차이를 강조한 예제
아래 예제를 통해 두 방식의 동작 차이를 확인할 수 있습니다:


from collections import defaultdict

# Using defaultdict
graph_default = defaultdict(list)
graph_default[0].append(1)  # 자동으로 빈 리스트 초기화 후 1 추가
print(graph_default[0])  # 출력: [1]
print(graph_default[1])  # 출력: [] (키 1에 접근하면 자동으로 빈 리스트 생성)

# Using {}
graph_dict = {}
graph_dict[0] = []
graph_dict[0].append(1)
print(graph_dict[0])  # 출력: [1]
# print(graph_dict[1])  # KeyError 발생 (키 1이 존재하지 않음)